In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import requests
import operator
import itertools

from ipywidgets import widgets
from IPython.display import display, HTML, FileLink

In [2]:
def create_start_time(row):
    date = row["Arrival"]
    if row.Starttime is None:
        return None
    
    return pd.Timestamp(date.year, date.month, date.day, int(row.Starttime[0:2]), int(row.Starttime[2:]))

def create_end_time(row):
    date = row["Arrival"]
    if row.Endtime is None:
        return None
    
    return pd.Timestamp(date.year, date.month, date.day, int(row.Endtime[0:2]), int(row.Endtime[2:]))

In [3]:
def get_date(df, date):
    """Return the field trip entries for the given date."""
    
    if isinstance(date, str):
        split = date.split('-')
        date = datetime.datetime(int(split[0]),int(split[1]),int(split[2])).date()

    return df[df.Arrival.dt.date == pd.Timestamp(date).date()]

def get_date_range(df, start, end):
    """Return a DateFrame containing entries between start and end (inclusive)"""

    if isinstance(start, str):
        split = start.split('-')
        start = datetime.datetime(int(split[0]),int(split[1]),int(split[2])).date()
    if isinstance(end, str):
        split = end.split('-')
        end = datetime.datetime(int(split[0]),int(split[1]),int(split[2])).date()

    return df[(df.Arrival.dt.date >= pd.Timestamp(start).date()) & (df.Arrival.dt.date <= pd.Timestamp(end).date())]


def get_admission(df, date):
    """Return a tuple containing the number of schools and visitors for the day."""

    day = get_date(df, date)
    admission = day[day.Category == 'Admission']

    by_school = admission.groupby('Address').sum(numeric_only=True).reset_index()
    
    return (len(by_school), admission.sum(numeric_only=True).Quantity)

def get_location(df, location):
    """Return the field trip entries for the given location."""
    
    return df[df.Location == location]

def decimal_time(date):
    "Return the time as a decimal number of hours"
    
    return date.hour + date.minute/60

In [4]:
def format_name(name):
    """Format the name of a given program."""
    
    if name == 'SCH - L - Amusement Park Physics STEM Lab':
        return "Amusement\nPark Physics"
    if name == 'SCH - L - Squid Dissection STEM Lab':
        return "Squid\nDissection"
    if name == 'SCH - D - Matter Matters':
        return "Matter Matters"
    if name == 'PS - School Shows':
        return "School Show"
    if name == 'SCH - D - Cooking Up a Storm':
        return "Cooking Up\na Storm"
    if name == 'SCH - L - Cow Eye Dissection STEM Lab':
        return "Cow Eye\nDissection"
    if name == 'SCH - D - Get Energized!':
        return 'Get Energized!'
    if name == 'PS - To Worlds Beyond':
        return 'To Worlds\nBeyond'
    if name == 'SCH - L - Splitting Molecules STEM Lab':
        return 'Splitting\nMolecules'
    if name == 'SCH - D - Space Exploration':
        return 'Space\nExploration'
    if name == 'SCH - L - Fetal Pig STEM Lab':
        return 'Fetal Pig\nDissection'
    if name == 'PS - Nightwatch':
        return 'Nightwatch'
    if name == 'SCH - D - Chemistry is a Blast!':
        return 'Chemistry\nis a Blast'
    if name == "SCH - D - Shocking, It's Science!":
        return "Shocking,\nIt's Science!"
    if name == "SCH - D - Get Fired Up!":
        return 'Get Fired Up!'
    
    return name

def get_color(name_colors, name):
    """Return a color for each unique school name"""
    
    if name not in name_colors:
         name_colors[name] = sb.color_palette("pastel", n_colors=10)[len(name_colors)]

    return name_colors[name]

def check_legend(legend_names, name):
    """Check if the given name has been added to the legend."""
    
    if name in legend_names:
        return None
    
    legend_names[name] = True
    return name

def generate_schedule(date):
    """Generate a schedule image and return it."""
    
    locations = {
        "Jack Wood Hall": 1,
        "Eureka Theater": 2,
        "Learning Lab": 3,
        "Green Classroom": 4,
        "Yellow Classroom": 5,
        "Sudekum Planetarium": 6
    }
    
    day = get_date(data, date)

    if len(day) == 0:
        return
        
    name_colors = {}
    legend_names = {}
    
    plt.clf()
    combo = day.groupby(["Name", "Program", "Location",  "Start time", "End time", "Capacity"]).sum(numeric_only=True).reset_index()
    
    for i, row in combo.iterrows():
        if row.Location is None:
            continue
    
        start = decimal_time(row["Start time"])
        end = decimal_time(row["End time"])
        duration = end - start
        plt.bar(locations[row.Location], duration, bottom=start, label=check_legend(legend_names, row.Name), color=get_color(name_colors, row.Name))
        plt.text(locations[row.Location], (start + end)/2, format_name(row.Program) + "\n("+str(row.Quantity)+"/" + str(row.Capacity)+ ")", ha='center', va='center', wrap=True)
    
    # Add public shows
    plt.bar(2, .5, bottom=12.5, color=(0.5, 0.5, 0.5))
    plt.text(2, 12.75, "Live Science", ha='center', va='center', wrap=True, color='white')
    
    plt.bar(6, .5, bottom=11.5, color=(0.5, 0.5, 0.5))
    plt.text(6, 11.75, "Public Show", ha='center', va='center', wrap=True, color='white')
    
    plt.bar(6, .5, bottom=13.25, color=(0.5, 0.5, 0.5))
    plt.text(6, 13.5, "Public Show", ha='center', va='center', wrap=True, color='white')
    
    plt.bar(6, .5, bottom=14.25, color=(0.5, 0.5, 0.5))
    plt.text(6, 14.5, "Public Show", ha='center', va='center', wrap=True, color='white')
    
    
    plt.legend(bbox_to_anchor=(0.5, -0.2), loc='lower center', ncol=2)
    
    plt.title("Field Trip Schedule: " + str(np.min(day.Arrival.dt.date)))
    plt.gca().invert_yaxis();
    plt.yticks([9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15], 
               ["9 AM", "9:30 AM", "10 AM", "10:30 AM", "11 AM", "11:30 AM",  "12 PM", "12:30 PM", "1 PM", "1:30 PM", "2 PM", "2:30 PM", "3 PM"]);
    plt.xticks([1, 2, 3, 4, 5, 6], ["Jack Wood\nHall", "Eureka\nTheater", "Learning\nLab", "Green\nClassroom", "Yellow\nClassroom", "Sudekum\nPlanetarium"]);
    plt.grid(which='major', axis='y', zorder=0)
    
    fig = plt.gcf()
    fig.set_size_inches(10, 8)
    plt.tight_layout()

    return plt.gcf()
    # plt.savefig('schedules/' + str(date) + '.pdf', dpi=300)

def reset_search_schedule():
    """Rebuild the base schedule with no entries"""

    global schedule_dict

    schedule_dict = {}

    today = datetime.datetime.now().date()
    next_year = today + pd.Timedelta('365 d')

    day_dict = {9: False, 9.25: False, 9.5: False, 9.75: False,
                10: False, 10.25: False, 10.5: False, 10.75: False,
                11: False, 11.25: False, 11.5: False, 11.75: False,
                12: False, 12.25: False, 12.5: False, 12.75: False,
                13: False, 13.25: False, 13.5: False, 13.75: False,
                14: False, 14.25: False, 14.5: False, 14.75: False}
    jwh_dict = {9: True, 9.25: True, 9.5: True, 9.75: True,
                10: False, 10.25: False, 10.5: False, 10.75: False,
                11: False, 11.25: False, 11.5: False, 11.75: False,
                12: False, 12.25: False, 12.5: False, 12.75: False,
                13: False, 13.25: False, 13.5: False, 13.75: False,
                14: True, 14.25: True, 14.5: True, 14.75: True}
    eureka_dict = {9: False, 9.25: False, 9.5: False, 9.75: False,
                10: False, 10.25: False, 10.5: False, 10.75: False,
                11: False, 11.25: False, 11.5: False, 11.75: False,
                12: False, 12.25: True, 12.5: True, 12.75: True,
                13: False, 13.25: False, 13.5: False, 13.75: False,
                14: False, 14.25: False, 14.5: False, 14.75: False}
    planet_dict = {9: False, 9.25: False, 9.5: False, 9.75: False,
                10: False, 10.25: False, 10.5: False, 10.75: False,
                11: False, 11.25: False, 11.5: True, 11.75: True,
                12: True, 12.25: True, 12.5: False, 12.75: False,
                13: True, 13.25: True, 13.5: True, 13.75: True,
                14: True, 14.25: True, 14.5: True, 14.75: True}
    
    for date in pd.date_range(today, next_year):
        if date.weekday() not in [0, 3, 4]:
            # Select Mon, Th, Fri
            continue
        if date.date().month in [6, 7, 8]:
            # Ignore summer
            continue
            
        date_str = str(date.date())
        admission = get_admission(data, date.date())
        schedule_dict[date_str] = {
            'Admission': {'groups': admission[0], 'quantity': admission[1]},
            'Jack Wood Hall': jwh_dict.copy(),
            "Eureka Theater": eureka_dict.copy(),
            "Learning Lab": day_dict.copy(),
            "Green Classroom": day_dict.copy(),
            "Yellow Classroom": day_dict.copy(),
            "Sudekum Planetarium": planet_dict.copy()
        }    

def build_search_schedule():
    """From the data, build a dict representing the daily schedule"""

    global schedule_dict

    reset_search_schedule()

    for i, row in data.iterrows():
        if row.Location is None or row.Location == 'Admission':
            continue

        date = str(row.Arrival.date())
        if date not in schedule_dict:
            continue

        start_time = decimal_time(row['Start time'].time())
        end_time = decimal_time(row['End time'].time())
        # Iterate the schedule in the given date/location and block matching times
        for slot in schedule_dict[date][row.Location]:
            if slot >= start_time and slot < end_time:
                schedule_dict[date][row.Location][slot] = True

def search_admission(number):
    """Search the schedule for dates that have capacity for the given number."""

    results = {}
    
    for date in schedule_dict:
        admission = schedule_dict[date]['Admission']
        
        if (6 - admission['groups']) > 0 and (600 - admission['quantity']) >= number:
            results[date] = True

    return results

def search_schedule(location, duration, start_time=9, end_time=14):
    """Search the schedule for gaps matching the given location and duration."""
    results = {}
    
    for date in schedule_dict:
        loc_schedule = schedule_dict[date][location]
        # Find the available gaps
        for slot in loc_schedule:
            if loc_schedule[slot] is True:
                # This slot is full
                continue
            if slot < start_time or slot >= end_time:
                # This time is outside of their visit
                continue
            i = 0
            gap = 0
            while slot + i*0.25 in loc_schedule and loc_schedule[slot + i*0.25] is False and slot + i*0.25 < end_time:
                gap += 0.25
                i += 1
            if gap >= duration:
                if date not in results:
                    results[date] = {location: {}}
                results[date][location][slot] = gap
    return results

def combo_search(criteria, number=365, start_date=None, end_date=None, start_time=9, end_time=14):
    """Search the schedule for multiple critera, given by (location, duration)."""

    if isinstance(start_date, str):
        split = start_date.split('-')
        start_date = datetime.datetime(int(split[0]),int(split[1]),int(split[2])).date()
    if isinstance(end_date, str):
        split = end_date.split('-')
        end_date = datetime.datetime(int(split[0]),int(split[1]),int(split[2])).date()
        
    combo_results = {}

    i = 0
    admission = False
    group_size = 0
    criteria_dict = {}
    for criterion in criteria:
        if criterion[0] != 'Admission':
            criteria_dict[criterion[0]] = criterion[1]
            combo_results[i] = search_schedule(criterion[0], criterion[1], start_time=start_time, end_time=end_time)
            i += 1
        else:
            admission = True
            group_size = criterion[1]
    admission_match = search_admission(group_size)

    if len(combo_results) == 0:
        # Bail out without anything to match
        output_dict = admission_match
        
        # Cut down the dictionary to the given number of elements. 
        return {k: output_dict[k] for k, _ in zip(output_dict, range(number))}
        
    match_dict = {}
    # Throw out dates that are not in all result sets and match the date constraints
    for date in combo_results[0]:
        bad_match = False
        for key in combo_results:
            results_dict = combo_results[key]
            if date not in results_dict:
                bad_match = True
        if date not in admission_match:
            bad_match = True
        if start_date is not None and pd.to_datetime(date).date() < start_date:
            bad_match = True
        if end_date is not None and pd.to_datetime(date).date() > end_date:
            bad_match = True
        if bad_match is False:
            match_dict[date] = True

    # Brute force the various options to find a schedule that match all criteria at different times.
    jigsaw_dict = {}
    for date in match_dict:
        subdict = {}
        for index in combo_results:
            result_dict = combo_results[index][date]
            location = list(result_dict.keys())[0]
            options = list(result_dict[location].keys())
            subdict[location] = {'duration': criteria_dict[location], 'options': options}
        jigsaw_dict[date] = subdict

    output_dict = jigsaw_schedule(jigsaw_dict)
    
    # Cut down the dictionary to the given number of elements. 
    return {k: output_dict[k] for k, _ in zip(output_dict, range(number))}

def jigsaw_schedule(options_dict):
    """Brute force a series of schedule options to find a set that matches."""

    result_dict = {}
    slots_template = {9: False, 9.25: False, 9.5: False, 9.75: False,
                 10: False, 10.25: False, 10.5: False, 10.75: False,
                 11: False, 11.25: False, 11.5: False, 11.75: False,
                 12: False, 12.25: False, 12.5: False, 12.75: False,
                 13: False, 13.25: False, 13.5: False, 13.75: False,
                 14: False, 14.25: False, 14.5: False, 14.75: False}

    for date in options_dict:
        locations = options_dict[date]
        option_lists = []
        for location in locations:
            location_options = []
            duration = locations[location]["duration"]
            options = locations[location]["options"]
            for option in options:
                location_options.append((location, option, duration))
            option_lists.append(location_options)
        permutations = list(itertools.product(*option_lists))
        match = None
        for perm in permutations:
            slots = slots_template.copy()
            error = False
            for item in perm:
                location, start, duration = item
                for slot in slots:
                    if slot >= start and slot < start + duration:
                        if slots[slot] is True:
                            error = True
                        else:
                            slots[slot] = True
            if error is False:
                match = perm
                break
        if match is not None:
            result_dict[date] = match
    return result_dict

def visualize_search_schedule(date, overlays=[]):
    """Create a schedule graphic that shows the time slots available on a given day."""

    locations = {
        "Jack Wood Hall": 1,
        "Eureka Theater": 2,
        "Learning Lab": 3,
        "Green Classroom": 4,
        "Yellow Classroom": 5,
        "Sudekum Planetarium": 6
    }
    
    day = schedule_dict[date]

    if len(day) == 0:
        return
    
    plt.clf()
    
    for location in day:
        for slot in day[location]:
            if day[location][slot] is True:
                plt.bar(locations[location], 0.25, bottom=slot, color=(0.5, 0.5, 0.5))

    # Add overlays
    for overlay in overlays:
        location, start, duration = overlay
        plt.bar(locations[location], duration, bottom=start)
    
    plt.title("Field Trip Availability: " + date)
    plt.gca().invert_yaxis();
    plt.yticks([9, 9.5, 10, 10.5, 11, 11.5, 12, 12.5, 13, 13.5, 14, 14.5, 15], 
               ["9 AM", "9:30 AM", "10 AM", "10:30 AM", "11 AM", "11:30 AM",  "12 PM", "12:30 PM", "1 PM", "1:30 PM", "2 PM", "2:30 PM", "3 PM"]);
    plt.xticks([1, 2, 3, 4, 5, 6], ["Jack Wood\nHall", "Eureka\nTheater", "Learning\nLab", "Green\nClassroom", "Yellow\nClassroom", "Sudekum\nPlanetarium"]);
    plt.grid(which='major', axis='y', zorder=0)

    fig = plt.gcf()
    fig.set_size_inches(10, 8)
    plt.tight_layout()

    return plt.gcf()


In [5]:
def generate_schedule_from_browser(*args):
    """Use the date from the date picker to create a schedule"""
    display(browse_date_picker.value)
    
    browse_output.clear_output()
    with browse_output:
        display(generate_schedule(browse_date_picker.value))

def search_from_browser(*args):
    """Collect inputs from the find tab and search for a matching schedule slot."""

    if find_start_date_picker.value is not None:
        start_date = find_start_date_picker.value
    else:
        start_date = datetime.datetime.now().date()
    if find_end_date_picker.value is not None:
        end_date = find_end_date_picker.value
    else:
        end_date = (datetime.datetime.now() + pd.Timedelta('90 d')).date()
    if find_start_time_picker.value is not None:
        start_time = decimal_time(find_start_time_picker.value)
    else:
        start_time = 9.5
    if find_end_time_picker.value is not None:
        end_time = decimal_time(find_end_time_picker.value)
    else:
        end_time = 13.5

    criteria = []
    if find_programs_demo.value > 0:
        criteria.append(('Eureka Theater', find_programs_demo.value))
    if find_programs_lab.value > 0:
        criteria.append(('Learning Lab', find_programs_lab.value))
    if find_programs_planet.value > 0:
        criteria.append(('Sudekum Planetarium', find_programs_planet.value))
    if find_misc_lunch.value > 0:
        criteria.append(('Jack Wood Hall', find_misc_lunch.value))
    if find_misc_visitors.value > 0:
        criteria.append(('Admission', find_misc_visitors.value))

    results = combo_search(criteria,
                         start_date=start_date,
                         end_date=end_date,
                         start_time=start_time, end_time=end_time,
                         number=1)
    find_output.clear_output()
    with find_output:
        for date in results:
            overlays =results[date]
            if not isinstance(overlays, tuple):
                overlays = []
            display(visualize_search_schedule(date, overlays));

In [6]:
username = ''
password = ''
data = pd.DataFrame()
schedule_dict = {}

In [7]:
def login(*args):
    """Get the username and password from the login fields and retrieve the data."""

    global username, password

    pw_status.value = username
    username = user_field.value
    password = pw_field.value

    pw_status.value = "Loading..."
        
    retrieve_data()
    show_interface()

def retrieve_data():
    """Retrieve the latest data from the server"""

    global data
    
    url = "https://s20aalt05web01.sky.blackbaud.com/2532Altru/ODataQuery.ashx?databasename=d32a1a5b-211a-4f58-bab1-36132004843f&AdHocQueryID=379fdd07-ded0-4a38-b183-7c0e49118619"

    session = requests.Session()
    session.auth = (username, password)
    
    r = session.get(url)

    data = pd.DataFrame(r.json()['value'])

    data["Arrival"] = pd.to_datetime(data["Arrival"])
    data["Departure"] = pd.to_datetime(data["Departure"])
    
    data["Start time"] = data.apply(create_start_time, axis=1)
    data["End time"] = data.apply(create_end_time, axis=1)
    data["Ticket type"] = data["Tickettype"]
    
    data = data[["Name", "Arrival", "Departure", "Program", "Category", "Location", "Ticket type", "Quantity", "Capacity", "Start time", "End time", "Address"]]

    build_search_schedule()
    login_output.clear_output()
    

In [8]:
user_label = widgets.Label('Username:', layout=widgets.Layout(width='75px'))
user_field = widgets.Text(layout=widgets.Layout(width='175px'))
pw_field = widgets.Password(layout=widgets.Layout(width='175px'))
pw_submit_button = widgets.Button(description='Login',layout=widgets.Layout(width='75px'))
pw_submit_button.on_click(login)
pw_status = widgets.Label('', layout=widgets.Layout(width='150px'))
pw_label = widgets.Label('Password:', layout=widgets.Layout(width='75px'))

pw_login_box = widgets.VBox([
    widgets.HBox([user_label, user_field],
                layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap')),
    widgets.HBox([pw_label,pw_field], 
                 layout=widgets.Layout(width='100%',display='inline-flex',flex_flow='row wrap')),
    widgets.HBox([ pw_submit_button, pw_status])
])

login_output = widgets.Output()
display(login_output)

login_output = widgets.Output()
display(HTML('<h1>ASC Field Trip Planner</h1>'))
display(login_output)

with login_output:
    display(pw_login_box)

Output()

Output()

In [9]:
browse_date_picker = widgets.DatePicker()
browse_select_date_button = widgets.Button(description="Select")
browse_select_date_button.on_click(generate_schedule_from_browser)
browse_date_box = widgets.HBox([browse_date_picker,browse_select_date_button])

browse_output = widgets.Output(layout={'border': '1px solid black'})


find_start_date_picker = widgets.DatePicker(description="Start date")
find_end_date_picker = widgets.DatePicker(description="End date")
find_date_box = widgets.HBox([find_start_date_picker,find_end_date_picker])
find_start_time_picker = widgets.TimePicker(description="Arrival")
find_end_time_picker = widgets.TimePicker(description="Departute")
find_time_box = widgets.HBox([find_start_time_picker,find_end_time_picker])
find_datetime_box = widgets.VBox([find_date_box, find_time_box])

find_programs_demo = widgets.Dropdown(
    options=[('No', 0), ('Short', 0.5), ('Long', 1)],
    value=0,
    description='Demo',
)
find_programs_lab = widgets.Dropdown(
    options=[('No', 0), ('Short', 0.5), ('Long', 1)],
    value=0,
    description='Lab',
)
find_programs_planet = widgets.Dropdown(
    options=[('No', 0), ('Yes', 0.5)],
    value=0,
    description='Planetarium',
)
find_programs_box = widgets.HBox([find_programs_demo, find_programs_lab, find_programs_planet])

find_misc_lunch = widgets.Dropdown(
    options=[('No', 0), ('Yes', 0.5)],
    value=0,
    description='Lunch',
)
find_misc_visitors = widgets.IntText(description="Visitors")
find_misc_box = widgets.HBox([find_misc_lunch, find_misc_visitors])

find_search = widgets.Button(description="Search")
find_search.on_click(search_from_browser)

find_output = widgets.Output(layout={'border': '1px solid black'})


interface = widgets.Tab(layout=widgets.Layout(width="500px"))
interface.children = [
     widgets.VBox([browse_date_box, browse_output]),
     widgets.VBox([find_datetime_box, find_programs_box, find_misc_box, find_search, find_output])
]
interface.titles = ["Schedule browser", "Booking helper"]

def show_interface(*args):
    display(interface)

In [10]:
show_interface()

In [11]:
test = combo_search([('Eureka Theater', 1),
                    ('Jack Wood Hall', .5),
                    ],
                     start_date='2024-05-01',
                     end_date='2024-05-30',
                     start_time=10, end_time=13,
                     number=5)

In [12]:
search_schedule('Eureka Theater', 1, start_time=10, end_time=13)

{}

In [13]:
a = {1: 10, 2: 20, 3: 5}